# Azure SQLDB Vector Store

Azure SQL provides a dedicated [Vector data type](https:\learn.microsoft.com\sql\t-sql\data-types\vector-data-type?view=azuresqldb-current&viewFallbackFrom=sql-server-ver16&tabs=csharp-sample) that simplifies the creation, storage, and querying of vector embeddings directly within a relational database. This eliminates the need for separate vector databases and related integrations, increasing the security of your solutions while reducing the overall complexity.

Azure SQL is a robust service that combines scalability, security, and high availability, providing all the benefits of a modern database solution. It leverages a sophisticated query optimizer and enterprise features to perform vector similarity searches alongside traditional SQL queries, enhancing data analysis and decision-making.  
  
Read more on using [Intelligent applications with Azure SQL Database](https://learn.microsoft.com/azure/azure-sql/database/ai-artificial-intelligence-intelligent-applications?view=azuresql)

This notebook shows you how to leverage this integrated SQL [vector database](https:\devblogs.microsoft.com\azure-sql\exciting-announcement-public-preview-of-native-vector-support-in-azure-sql-database\) to store documents and perform vector search queries using Cosine (cosine distance), L2 (Euclidean distance), and IP (inner product) to locate documents close to the query vectors

Install the `langchain-sqlserver` python package.

The code lives in an integration package called:[langchain-sqlserver](https://github.com/langchain-ai/langchain-azure/tree/main/libs/sqlserver).

In [ ]:
!pip install langchain-sqlserver==0.1.1

<span style="color: var(--vscode-foreground);">Find your Azure SQL DB connection string in the Azure portal under your database settings</span>

For more info: [Connect to Azure SQL DB - Python](https:\learn.microsoft.com\en-us\azure\azure-sql\database\connect-query-python?view=azuresql)

In [11]:
import pyodbc
import os

#Define your SQLServer Connection String
_CONNECTION_STRING = (
    'Driver={ODBC Driver 18 for SQL Server};'
    'Server=<YOUR_DBSERVER>.database.windows.net,1433;'
    'Database=test;'
    'TrustServerCertificate=yes;'
    'Connection Timeout=60;'
    'LongAsMax=yes;'
)

# Connection string can vary:
# "mssql+pyodbc://<username>:<password><servername>/<dbname>?driver=ODBC+Driver+18+for+SQL+Server" -> With Username and Password specified
# "mssql+pyodbc://<servername>/<dbname>?driver=ODBC+Driver+18+for+SQL+Server&Trusted_connection=yes" -> Uses Trusted connection
# "mssql+pyodbc://<servername>/<dbname>?driver=ODBC+Driver+18+for+SQL+Server" -> Uses EntraID connection
# "mssql+pyodbc://<servername>/<dbname>?driver=ODBC+Driver+18+for+SQL+Server&Trusted_connection=no" -> Uses EntraID connection



In this example we use Azure OpenAI to generate embeddings , however you can use different embeddings provided in LangChain.

You can deploy a version of Azure OpenAI instance on Azure Portal following this [guide](https:\learn.microsoft.com\en-us\azure\ai-services\openai\how-to\create-resource?pivots=web-portal). <span style="color: var(--vscode-foreground);">Once you have your instance running, make sure you have the name of your instance and key. You can find the key in the Azure Portal, under the "Keys and Endpoint" section of your instance.</span>

In [ ]:
!pip install langchain-openai

In [ ]:
#Import the necessary Libraries
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI

#Set your AzureOpenAI details 
azure_endpoint = "https://<YOUR_ENDPOINT>.openai.azure.com/"
azure_deployment_name_embedding = "text-embedding-3-small"
azure_deployment_name_chatcompletion = "chatcompletion"
azure_api_version = "2023-05-15"
azure_api_key = "YOUR_KEY"


# Use AzureChatOpenAI for chat completions
llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_deployment_name_chatcompletion,
    openai_api_version=azure_api_version,
    openai_api_key=azure_api_key
)

# Use AzureOpenAIEmbeddings for embeddings
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_deployment_name_embedding, 
    openai_api_version=azure_api_version,
    openai_api_key=azure_api_key
)

## Creating Azure SQL DB Vector Search

In [15]:
from langchain_sqlserver import SQLServer_VectorStore
from langchain_community.vectorstores.utils import DistanceStrategy

# Initialize the vector store 
vector_store = SQLServer_VectorStore(
    connection_string=_CONNECTION_STRING,
    distance_strategy=DistanceStrategy.COSINE, #optional, if not provided, defaults to COSINE
    embedding_function=embeddings, # you can use different embeddings provided in LangChain
    embedding_length=1536,
    table_name="langchain_test_table" # using table with a custom name
)

## Add items to vector store

In [16]:
## we will use some artificial data for this example
query = [
    "I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.",
    "The candy is just red , No flavor . Just  plan and chewy .  I would never buy them again",
    "Arrived in 6 days and were so stale i could not eat any of the 6 bags!!",
    "Got these on sale for roughly 25 cents per cup, which is half the price of my local grocery stores, plus they rarely stock the spicy flavors. These things are a GREAT snack for my office where time is constantly crunched and sometimes you can't escape for a real meal. This is one of my favorite flavors of Instant Lunch and will be back to buy every time it goes on sale.",
    "If you are looking for a less messy version of licorice for the children, then be sure to try these!  They're soft, easy to chew, and they don't get your hands all sticky and gross in the car, in the summer, at the beach, etc. We love all the flavos and sometimes mix these in with the chocolate to have a very nice snack! Great item, great price too, highly recommend!",
    "We had trouble finding this locally - delivery was fast, no more hunting up and down the flour aisle at our local grocery stores.",
    "Too much of a good thing? We worked this kibble in over time, slowly shifting the percentage of Felidae to national junk-food brand until the bowl was all natural. By this time, the cats couldn't keep it in or down. What a mess. We've moved on.",
    "Hey, the description says 360 grams - that is roughly 13 ounces at under $4.00 per can. No way - that is the approximate price for a 100 gram can.",
    "The taste of these white cheddar flat breads is like a regular cracker - which is not bad, except that I bought them because I wanted a cheese taste.<br /><br />What was a HUGE disappointment? How misleading the packaging of the box is. The photo on the box (I bought these in store) makes it look like it is full of long flatbreads (expanding the length and width of the box). Wrong! The plastic tray that holds the crackers is about 2"
    " smaller all around - leaving you with about 15 or so small flatbreads.<br /><br />What is also bad about this is that the company states they use biodegradable and eco-friendly packaging. FAIL! They used a HUGE box for a ridiculously small amount of crackers. Not ecofriendly at all.<br /><br />Would I buy these again? No - I feel ripped off. The other crackers (like Sesame Tarragon) give you a little<br />more bang for your buck and have more flavor.",
    "I have used this product in smoothies for my son and he loves it. Additionally, I use this oil in the shower as a skin conditioner and it has made my skin look great. Some of the stretch marks on my belly has disappeared quickly. Highly recommend!!!",
    "Been taking Coconut Oil for YEARS.  This is the best on the retail market.  I wish it was in glass, but this is the one.",
]

query_metadata = [
    {"id": 1, "summary": "Good Quality Dog Food"},
    {"id": 8, "summary": "Nasty No flavor"},
    {"id": 4, "summary": "stale product"},
    {"id": 11, "summary": "Great value and convenient ramen"},
    {"id": 5, "summary": "Great for the kids!"},
    {"id": 2, "summary": "yum falafel"},
    {"id": 9, "summary": "Nearly killed the cats"},
    {"id": 6, "summary": "Price cannot be correct"},
    {"id": 3, "summary": "Taste is neutral, quantity is DECEITFUL!"},
    {"id": 7, "summary": "This stuff is great"},
    {"id": 10, "summary": "The reviews don't lie"},
]


In [19]:
vector_store.add_texts(texts=query, metadatas=query_metadata)

[1, 8, 4, 11, 5, 2, 9, 6, 3, 7, 10]

## Querying Data:
Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent.

Performing a simple similarity search can be done as follows:

In [28]:
# Perform a similarity search between the embedding of the query and the embeddings of the documents
simsearch_result = vector_store.similarity_search("Good reviews", k = 3)
print(simsearch_result)


[Document(metadata={'id': 1, 'summary': 'Good Quality Dog Food'}, page_content='I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'), Document(metadata={'id': 7, 'summary': 'This stuff is great'}, page_content='I have used this product in smoothies for my son and he loves it. Additionally, I use this oil in the shower as a skin conditioner and it has made my skin look great. Some of the stretch marks on my belly has disappeared quickly. Highly recommend!!!'), Document(metadata={'id': 5, 'summary': 'Great for the kids!'}, page_content="If you are looking for a less messy version of licorice for the children, then be sure to try these!  They're soft, easy to chew, and they don't get your hands all sticky and gross in the car, in the summer, at the beach, etc. We love all the fla

## Filtering Support:

The vectorstore supports a set of filters that can be applied against the metadata fields of the documents.This feature enables developers and data analysts to refine their queries, ensuring that the search results are accurately aligned with their needs. By applying filters based on specific metadata attributes, users can limit the scope of their searches, concentrating only on the most relevant data subsets.

In [29]:
# hybrid search -> filter for cases where id not equal to 1.
hybrid_simsearch_result = vector_store.similarity_search("Good reviews", k=3, filter={"id": {"$ne": 1}})
print(hybrid_simsearch_result)

[Document(metadata={'id': 7, 'summary': 'This stuff is great'}, page_content='I have used this product in smoothies for my son and he loves it. Additionally, I use this oil in the shower as a skin conditioner and it has made my skin look great. Some of the stretch marks on my belly has disappeared quickly. Highly recommend!!!'), Document(metadata={'id': 5, 'summary': 'Great for the kids!'}, page_content="If you are looking for a less messy version of licorice for the children, then be sure to try these!  They're soft, easy to chew, and they don't get your hands all sticky and gross in the car, in the summer, at the beach, etc. We love all the flavos and sometimes mix these in with the chocolate to have a very nice snack! Great item, great price too, highly recommend!"), Document(metadata={'id': 3, 'summary': 'Taste is neutral, quantity is DECEITFUL!'}, page_content='The taste of these white cheddar flat breads is like a regular cracker - which is not bad, except that I bought them beca

## Similarity Search with Score:
If you want to execute a similarity search and receive the corresponding scores you can run:

In [30]:
simsearch_with_score_result = vector_store.similarity_search_with_score("Not a very good product", k=12)
print(simsearch_with_score_result)

[(Document(metadata={'id': 3, 'summary': 'Taste is neutral, quantity is DECEITFUL!'}, page_content='The taste of these white cheddar flat breads is like a regular cracker - which is not bad, except that I bought them because I wanted a cheese taste.<br /><br />What was a HUGE disappointment? How misleading the packaging of the box is. The photo on the box (I bought these in store) makes it look like it is full of long flatbreads (expanding the length and width of the box). Wrong! The plastic tray that holds the crackers is about 2 smaller all around - leaving you with about 15 or so small flatbreads.<br /><br />What is also bad about this is that the company states they use biodegradable and eco-friendly packaging. FAIL! They used a HUGE box for a ridiculously small amount of crackers. Not ecofriendly at all.<br /><br />Would I buy these again? No - I feel ripped off. The other crackers (like Sesame Tarragon) give you a little<br />more bang for your buck and have more flavor.'), 0.651

For a full list of the different searches you can execute on a Azure SQL vector store, please refer to the [API reference](https://python.langchain.com/api_reference/sqlserver/index.html).

## Similarity Search when you already have embeddings you want to search on

In [32]:
# if you already have embeddings you want to search on
simsearch_by_vector = vector_store.similarity_search_by_vector([-0.0033353185281157494, -0.017689190804958344, -0.01590404286980629, -0.01751338131725788, -0.018054334446787834, 0.021841011941432953, -0.012313461862504482, -0.02273358590900898, -0.021286534145474434, -0.01814900152385235, 0.012252604588866234, 0.038759343326091766, 0.0015408731997013092, -0.00691406661644578, -0.013638799078762531, 0.024153590202331543, 0.039895348250865936, 0.0012036223197355866, 0.009372025728225708, -0.012178223580121994, -0.019853007048368454, 0.006024873349815607, 0.011319459415972233, -0.025167878717184067, -0.00759363966062665, 0.010284884832799435, 0.009831836447119713, -0.008492975495755672, -0.005639444105327129, -0.009446406736969948, 0.007444877177476883, -0.009277358651161194, -0.025289593264460564, -0.02119186706840992, -0.005906539969146252, -0.018906336277723312, -0.007539544254541397, -0.016066329553723335, -0.01171841286122799, -0.02093491330742836, 0.004608250688761473, 0.011042220517992973, 0.011549364775419235, -0.009541073814034462, 0.0025864355266094208, 0.0026202453300356865, -0.0036007240414619446, -0.011995651759207249, -0.02549245022237301, -0.007958783768117428, 0.015701185911893845, 0.016188044100999832, -0.005825396627187729, -0.00866878591477871, -0.00038881058571860194, -0.0006356207886710763, 0.0074110678397119045, 0.00766802066937089, -0.005419681314378977, -0.007674783002585173, 0.0086823096498847, -0.004740108270198107, -0.01406479999423027, 0.02170577272772789, -0.0029955320060253143, -0.008574118837714195, 0.005460252985358238, 0.0034130807034671307, -0.005521110258996487, 0.0045507741160690784, 0.03662257641553879, -0.004141678102314472, 0.004442583303898573, -0.0035026762634515762, 0.0021316963247954845, -0.009297644719481468, -0.032186754047870636, 0.01478156354278326, -0.0016355401603505015, -0.005835539661347866, 0.03253837302327156, -0.040301062166690826, -0.0090339295566082, -0.001611873391084373, 0.018703479319810867, -0.00610601669177413, -0.016323283314704895, 0.002358220750465989, -0.004118011333048344, 0.005784825421869755, 0.01579585298895836, 0.028886936604976654, -0.004111249465495348, 0.020475102588534355, -0.0360545739531517, 0.0018696717452257872, 0.0039658681489527225, 0.026723120361566544, -0.011177458800375462, -0.03759629279375076, 0.0012501105666160583, 0.007478686980903149, -0.03445876017212868, -0.011130125261843204, -0.019677195698022842, -0.004784060642123222, 0.01866290718317032, 0.0013667537132278085, 0.015417184680700302, -0.01467337366193533, -0.010109075345098972, 0.03976010903716087, 0.02482978254556656, -0.045196693390607834, -0.02562768943607807, -0.0017614809330552816, -0.0002624471380840987, -0.006839685142040253, 0.005757777485996485, -0.001704004593193531, 0.020650913938879967, 0.021841011941432953, 0.045845840126276016, 0.00012234854511916637, -0.0019170051673427224, 0.006082349922508001, -0.027940265834331512, 0.005524491425603628, -0.002109719906002283, 0.011393840424716473, 0.036784861236810684, 0.019257957115769386, 0.0018020524876192212, 0.006051921285688877, -0.01858176477253437, 0.021584058180451393, -0.0070459237322211266, 0.00785735435783863, -0.00133378931786865, -0.028048457577824593, -0.006944495253264904, 0.019744815304875374, -0.025587117299437523, -0.020556246861815453, 0.00877697579562664, 0.03251132741570473, 0.01059593353420496, 0.00782354548573494, 0.009493740275502205, 0.008236022666096687, 0.002108029555529356, -0.007999354973435402, 0.012692130170762539, -0.02217910811305046, 0.0018848860636353493, 0.021178342401981354, 0.00394896324723959, 0.02273358590900898, -0.006907304283231497, 0.014754516072571278, 0.007830306887626648, 0.012090318836271763, -0.0025661499239504337, 0.0011884080013260245, -0.005483919754624367, 0.03640619292855263, 0.022192630916833878, 0.00766802066937089, -0.004368202295154333, -0.0065353987738490105, -0.002111410489305854, 0.02401835098862648, 0.0008110081544145942, 0.012225557118654251, -0.01879814639687538, 0.020258720964193344, -0.009148881770670414, 0.00009382168354932219, -0.0399494431912899, -0.009764216840267181, -0.009656026028096676, -0.0017800763016566634, 0.03110484592616558, 0.029617223888635635, 0.0030479368288069963, -0.0005232038092799485, 0.017242904752492905, -0.02500559203326702, 0.01663433015346527, -0.0012703962856903672, 0.0030428653117269278, 0.010237551294267178, 0.031023703515529633, -0.01101517304778099, -0.6837656497955322, 0.0020471722818911076, -0.00041289994260296226, 0.007877640426158905, 0.018973955884575844, 0.020096436142921448, 0.006109397392719984, 0.007999354973435402, -0.018162526190280914, 0.0011757294414564967, -0.007803259417414665, 0.019190337508916855, -0.009669549763202667, -0.0013912656577304006, -0.0061127785593271255, 0.006575970444828272, 0.0009407525649294257, -0.013997181318700314, 0.004821251146495342, 0.012894987128674984, 0.01016993261873722, -0.0009990741964429617, -0.0026692692190408707, -0.008648499846458435, -0.008154879324138165, -0.018892813473939896, -0.0012788487365469337, -0.0019186957506462932, 0.0045981076546013355, 0.01714823767542839, -0.007018876262009144, 0.01300317794084549, -0.011427650228142738, 0.001988005358725786, 0.03518904745578766, -0.0015366470906883478, 0.003017508191987872, 0.0399494431912899, 0.010508028790354729, 0.051796332001686096, -0.017351094633340836, -0.018189573660492897, 0.015701185911893845, 0.00895954854786396, -0.024410542100667953, 0.0016237067757174373, 0.008161641657352448, 0.016323283314704895, 0.010629743337631226, -0.004929441958665848, -0.01866290718317032, 0.0015729924198240042, 0.015782328322529793, 0.002850150689482689, 0.003749486291781068, 0.006153350230306387, 0.023301586508750916, -0.02357206493616104, 0.024369971826672554, 0.0009737169602885842, 0.016539664939045906, 0.011813079938292503, -0.015579471364617348, -0.034648094326257706, -0.01755395159125328, 0.005051156505942345, -0.03202446922659874, 0.0069512571208179, 0.006437350995838642, -0.013402131386101246, 0.014889754354953766, 0.032781802117824554, -0.010413361713290215, -0.006369731388986111, 0.006417064927518368, 0.02078615128993988, 0.001138538820669055, -0.008323927409946918, -0.0180678591132164, -0.010000884532928467, -0.008560595102608204, -0.012536605820059776, -0.039246201515197754, 0.003942201379686594, 0.006295350380241871, 0.005460252985358238, -0.01954195834696293, -0.000006484578534582397, 0.011393840424716473, -0.007640973199158907, 0.002662507351487875, 0.03786677122116089, -0.007952021434903145, 0.0021993154659867287, -0.0058118728920817375, 0.022706538438796997, 0.0061702546663582325, 0.011522317305207253, 0.011278888210654259, -0.04157230257987976, -0.010589171200990677, 0.0006880256696604192, 0.027277598157525063, 0.0007429663091897964, 0.024951497092843056, 0.020961962640285492, -0.003031032159924507, 0.01539013721048832, 0.008161641657352448, -0.00791145022958517, 0.007086495403200388, 0.004026725422590971, 0.0037765339948236942, -0.01173193659633398, 0.003877962939441204, -0.02658788114786148, -0.005189775954931974, 0.009710121899843216, 0.024356447160243988, -0.030861416831612587, -0.0016406115610152483, -0.012976130470633507, 0.01461927779018879, -0.01729699969291687, 0.004926060792058706, 0.011143648996949196, -0.026357976719737053, -0.032592467963695526, 0.011123363859951496, -0.028156647458672523, -0.0021131010726094246, -0.01645852066576481, 0.02391016110777855, -0.025478925555944443, 0.018040811643004417, 0.01766214333474636, 0.019636625424027443, 0.013713180087506771, 0.00518639525398612, -0.010359265841543674, -0.014240610413253307, 0.01369965635240078, -0.00023413158487528563, -0.006859971210360527, 0.004253249615430832, -0.00883107166737318, -0.004320868756622076, -0.016201568767428398, -0.0029093173798173666, 0.012097080238163471, -0.00818868912756443, 0.005000442266464233, 0.0029279126320034266, 0.00014886796998325735, 0.029833605512976646, -0.028670554980635643, -0.006518493872135878, -0.005686777178198099, 0.00618039770051837, 0.010251075029373169, 0.01714823767542839, 0.019298529252409935, -0.024437589570879936, -0.0022720061242580414, -0.012103842571377754, -0.03540543094277382, 0.007032399997115135, 0.03359323367476463, -0.009493740275502205, -0.03562181070446968, 0.01555242296308279, -0.002633769065141678, -0.031753990799188614, 0.009466692805290222, 0.022084441035985947, 0.011684603057801723, 0.0029076270293444395, -0.025789974257349968, -0.001874743145890534, 0.011934794485569, 0.006846447009593248, -0.012678605504333973, -0.011400602757930756, -0.012381081469357014, 0.014524610713124275, 0.010785267688333988, 0.03635209798812866, 0.03851591423153877, -0.031077798455953598, 0.011265363544225693, -0.014429943636059761, 0.006532017607241869, 0.01411889586597681, 0.007133828941732645, 0.003715676721185446, -0.020475102588534355, 0.008283356204628944, 0.013530608266592026, 0.026804262772202492, 0.01571470871567726, 0.017905572429299355, -0.008364498615264893, 0.012719177640974522, -0.013091083616018295, 0.018230143934488297, -0.039976488798856735, 0.0017969810869544744, -0.019677195698022842, 0.016728997230529785, 0.012103842571377754, -0.01590404286980629, -0.013503560796380043, 0.0016617425717413425, -0.005700301378965378, 0.009737169370055199, 0.0218815840780735, -0.005507586523890495, 0.010305170901119709, 0.010406599380075932, -0.00954783521592617, -0.000835942744743079, -0.009405835531651974, 0.0008165022009052336, 0.004270154517143965, -0.006227731239050627, 0.014551658183336258, -0.002627007197588682, 0.03667667135596275, -0.004817870445549488, -0.010420123115181923, -0.02159758284687996, 0.004067296627908945, -0.0032355801668018103, 0.011921270750463009, 0.003877962939441204, 0.008006117306649685, 0.014889754354953766, -0.022206155583262444, 0.03359323367476463, -0.007755925878882408, 0.006095873657613993, 0.03264656662940979, 0.022828252986073494, -0.01755395159125328, 0.012462224811315536, 0.006488065235316753, 0.024951497092843056, -0.007282591424882412, -0.007952021434903145, -0.008012878708541393, -0.012861178256571293, -0.009047453291714191, -0.017783857882022858, 0.013462988659739494, 0.015484804287552834, -0.019406719133257866, 0.0219221543520689, 0.004243106581270695, 0.010934029705822468, 0.022977015003561974, 0.008371260948479176, 0.013429179787635803, 0.0024748637806624174, -0.002767316997051239, 0.03148351237177849, 0.004814489278942347, 0.005051156505942345, -0.02996884286403656, 0.013456227257847786, 0.015119659714400768, -0.015876995399594307, -0.0003748641174752265, 0.00811430811882019, -0.011373554356396198, 0.015620042569935322, 0.02346387319266796, -0.01836538314819336, 0.02038043551146984, 0.0040503921918570995, -0.01129241194576025, -0.038948677480220795, -0.04092315956950188, 0.023964256048202515, 0.015065564773976803, 0.016688426956534386, 0.007640973199158907, -0.035757049918174744, -0.014727468602359295, -0.001906862366013229, 0.020299293100833893, -0.0009948479710146785, 0.019636625424027443, 0.000619138590991497, 0.008946023881435394, -0.0012264437973499298, -0.004172106739133596, 0.03664962202310562, -0.006991828326135874, -0.000013986086742079351, -0.010203742422163486, 0.015430708415806293, -0.007958783768117428, -0.017026523128151894, -0.02684483490884304, -0.006481303367763758, 0.008837834000587463, -0.020610341802239418, -0.020921390503644943, -0.03018522448837757, 0.004178868606686592, -0.01785147748887539, 0.007891164161264896, -0.01533604133874178, -0.006373112555593252, -0.004882108420133591, 0.013523845933377743, -0.007323162630200386, -0.011062506586313248, 0.02564121223986149, -0.00813459325581789, -0.02251720428466797, -0.012482509948313236, -0.003969248849898577, -0.014281181618571281, 0.08265774697065353, 0.036162763833999634, -0.0014800159260630608, 0.029481984674930573, 0.021016057580709457, -0.011346506886184216, -0.020989010110497475, 0.0023193396627902985, 0.020137006416916847, -0.004405392799526453, 0.0056428248062729836, 0.0005396860069595277, 0.011332983151078224, -0.015376613475382328, 0.01555242296308279, 0.007444877177476883, -0.0005599717842414975, -0.010643267072737217, 0.016282711178064346, -0.008432118222117424, -0.004780679475516081, -0.018960433080792427, 0.0024004827719181776, 0.005761158652603626, -0.004124773200601339, -0.0139025142416358, 0.00620744563639164, 0.023599112406373024, -0.008973072282969952, -0.008601166307926178, -0.009412596933543682, 0.03540543094277382, -0.0007729723583906889, -0.0038982487749308348, 0.0037190576549619436, 0.006075588054955006, -0.00529796676710248, 0.015579471364617348, 0.04195097088813782, 0.0069208284839987755, 0.01421356201171875, 0.02508673444390297, -0.011447936296463013, -0.0036548194475471973, 0.021205391734838486, -0.025397783145308495, -0.008073735982179642, 0.03188923001289368, 0.0056800153106451035, -0.01307755894958973, 0.03416123613715172, 0.023233968764543533, -0.016404425725340843, -0.01954195834696293, 0.0044087739661335945, -0.007512496784329414, 0.01326013170182705, 0.003272770904004574, -0.0028907221276313066, -0.006433969829231501, -0.02823779173195362, -0.021732820197939873, 0.0006643589586019516, -0.007789735682308674, 0.005456871818751097, -0.04333040490746498, -0.023477397859096527, -0.004290440119802952, -0.020069388672709465, -0.006538779474794865, -0.024910924956202507, -0.011258602142333984, -0.010095551609992981, 0.017581000924110413, 0.015024993568658829, 0.010129360482096672, 0.028589410707354546, -0.0029228413477540016, 0.005855825264006853, 0.02112424746155739, -0.003303199540823698, -0.016512615606188774, 0.013807847164571285, -0.005889635067433119, -0.004905775189399719, 0.020907865837216377, -0.0023700541350990534, 0.009919741190969944, -0.006741637364029884, 0.005737491883337498, 0.02067796140909195, 0.0020184339955449104, 0.02280120551586151, 0.0014926944859325886, -0.0048787277191877365, 0.005345300305634737, 0.015876995399594307, 0.014254134148359299, -0.007701830472797155, 0.0032000800129026175, 0.02449168637394905, -0.02035338804125786, -0.032998185604810715, 0.002412316156551242, 0.0035533905029296875, 0.008296879939734936, 0.004834774881601334, -0.005629301071166992, 0.003272770904004574, 0.00027660492924042046, 0.017094140872359276, -0.014497563242912292, 0.015227850526571274, -0.004178868606686592, 0.014362324960529804, 0.012381081469357014, -0.008526785299181938, 0.017269952222704887, 0.002092815237119794, -0.02309872955083847, -0.024802733212709427, -0.015322517603635788, 0.042951736599206924, 0.032186754047870636, -0.01854119263589382, -0.005328395403921604, 0.0031764134764671326, 0.010136122815310955, 0.004358059260994196, 0.01461927779018879, 0.005598872434347868, 0.028156647458672523, -0.002091124653816223, -0.02111072465777397, -0.0181084293872118, -0.017026523128151894, 0.0034299856051802635, 0.016661379486322403, -0.01244870014488697, -0.0023886493872851133, -0.017635095864534378, -0.007864116691052914, 0.007282591424882412, -0.019677195698022842, -0.011576412245631218, -0.04995708912611008, -0.026357976719737053, -0.012807082384824753, 0.015701185911893845, 0.011725175194442272, -0.014240610413253307, -0.00020021632371935993, -0.009635740891098976, 0.01571470871567726, -0.0053351572714746, -0.033322758972644806, 0.0010252766078338027, 0.007140590809285641, 0.0012873010709881783, 0.04278944805264473, 0.024924449622631073, -0.00438848789781332, 0.01327365543693304, 0.020961962640285492, -0.01075821928679943, 0.004124773200601339, -0.00674839923158288, 0.005700301378965378, -0.019596053287386894, 0.010609457269310951, 0.015444232150912285, -0.00918269157409668, 0.023058157414197922, -0.0013380155432969332, -0.042032115161418915, 0.00910831056535244, 0.010906982235610485, -0.009757455438375473, -0.006616541650146246, -0.0024731734301894903, -0.004209297243505716, -0.003472247626632452, 0.014132419601082802, 0.01708061806857586, 0.005338538438081741, -0.00039451595512218773, -0.00031675383797846735, 0.020191103219985962, 0.006829542573541403, -0.0036852480843663216, 0.021381201222538948, -0.013665846548974514, 0.006126302294433117, -0.008662023581564426, -0.009419359266757965, -0.006427207961678505, -0.013692894019186497, 0.005710443947464228, 0.0032609375193715096, 0.010548599995672703, 0.02093491330742836, 0.022422537207603455, -0.0051728710532188416, 0.016147471964359283, -0.009906217455863953, -0.0016152544412761927, -0.015011469833552837, -0.0263985488563776, 0.012922035530209541, -0.028589410707354546, -0.01851414516568184, -0.027480455115437508, -0.027967313304543495, -0.02523549646139145, 0.000527007388882339, 0.008107545785605907, -0.005051156505942345, 0.0006026563933119178, 0.006707827560603619, -0.0032507944852113724, 0.0044662500731647015, 0.0012188366381451488, 0.005740872584283352, -0.007938497699797153, 0.028913984075188637, 0.03819134086370468, -0.007377258036285639, -0.015890520066022873, 0.023490920662879944, 0.012529843486845493, 0.005003822967410088, 0.012130890041589737, 0.0027537932619452477, 0.003877962939441204, -0.01795966736972332, 0.02196272648870945, 0.004111249465495348, -0.006562446244060993, -0.0453319326043129, 0.03743400797247887, 0.011894222348928452, -0.012070032767951488, -0.0011410745792090893, -0.015322517603635788, -0.03716352954506874, 0.025573592633008957, -0.024194160476326942, -0.010142885148525238, -0.03797496110200882, -0.0016237067757174373, -0.0206373892724514, 0.006461017765104771, -0.01582290045917034, 0.034215331077575684, 0.0005451800534501672, -0.004834774881601334, 0.0035128190647810698, -0.0020116721279919147, -0.002542483154684305, -0.02487035281956196, -0.00684982817620039, 0.034945618361234665, -0.018243668600916862, -0.005220204591751099, 0.01357117947191, 0.000029187207474024035, -0.015417184680700302, 0.009088024497032166, -0.02078615128993988, 0.029022173956036568, -0.025073211640119553, -0.013449464924633503, -0.00032731934334151447, -0.009419359266757965, 0.015268422663211823, 0.003908391576260328, -0.013483274728059769, -0.013158702291548252, -0.007728877943009138, -0.025898166000843048, 0.005108633078634739, 0.0037765339948236942, 0.0006935197161510587, -0.0006271683960221708, -0.02119186706840992, -0.0033505328465253115, -0.01571470871567726, -0.006802494637668133, -0.0036345336120575666, 0.012536605820059776, -0.003712295787408948, -0.011008410714566708, 0.007086495403200388, 0.002074219984933734, 0.005176252219825983, -0.0018121954053640366, 0.006038397550582886, 0.02031281776726246, -0.02812959998846054, 0.01659375987946987, -0.020989010110497475, 0.004327630624175072, -0.03951667994260788, -0.0003068222722504288, 0.008506499230861664, -0.0016177900834009051, -0.002733507426455617, 0.007546306122094393, -0.004611631389707327, 0.005135680548846722, -0.006897161714732647, 0.017567476257681847, 0.0023227205965667963, -0.013050511479377747, 0.01582290045917034, 0.003830629400908947, 0.010453932918608189, -0.03562181070446968, -0.034837428480386734, 0.02802141010761261, -0.006484684068709612, 0.016363853588700294, 0.0020319579634815454, -0.019961196929216385, -0.007627449464052916, -0.00048094178782776, 0.031862180680036545, 0.014943850226700306, -0.015457755886018276, -0.0232069194316864, -0.006396779324859381, 0.00875669065862894, -0.029481984674930573, -0.01468689739704132, -0.029941795393824577, -0.02523549646139145, -0.007877640426158905, 0.02530311606824398, 0.023585587739944458, -0.0030006032902747393, 0.02211148850619793, 0.016553187742829323, 0.0005071442574262619, -0.0021688868291676044, -0.021570535376667976, -0.035757049918174744, -0.008039926178753376, -0.012766511179506779, -0.016093377023935318, -0.009027167223393917, -0.016661379486322403, 0.02277415804564953, -0.0002588548813946545, -0.020515674725174904, -0.0070526860654354095, -0.004861822817474604, -0.01744576171040535, -0.002293982543051243, 0.00791145022958517, 0.025032639503479004, 0.013307464309036732, 0.00987916998565197, -0.006362969521433115, 0.016255663707852364, -0.000155946851009503, 0.04208621010184288, -0.0213406290858984, -0.023477397859096527, 0.01384841836988926, -0.02228729799389839, 0.0022212916519492865, -0.013983656652271748, 0.002599959494546056, -0.03026636876165867, 0.009074500761926174, 0.01630975864827633, 0.004425678867846727, 0.02641207166016102, 0.00650835083797574, -0.013395369984209538, -0.021638154983520508, 0.01946081407368183, 0.0038002007640898228, 0.0021688868291676044, 0.013402131386101246, -0.008858119137585163, -0.0139025142416358, 0.005115394946187735, -0.02934674732387066, -0.014091847464442253, 0.0019170051673427224, -0.0014808612177148461, -0.000039699883927823976, 0.01498442143201828, -0.01243517640978098, -0.013375083915889263, -0.02170577272772789, -0.03151056170463562, -0.010609457269310951, -0.002765626646578312, -0.013780799694359303, 0.030942561104893684, -0.002380196936428547, -0.014605754055082798, -0.01943376660346985, 0.0004572750476654619, -0.011272125877439976, -0.010501266457140446, 0.003881343873217702, -0.00335560436360538, 0.004909156356006861, -0.019190337508916855, 0.017107665538787842, -0.03786677122116089, 0.010832601226866245, 0.002527268836274743, 0.003763010259717703, 0.015511851757764816, 0.015890520066022873, 0.008912215009331703, -0.032403137534856796, -0.011738698929548264, -0.012590700760483742, -0.009845360182225704, -0.008161641657352448, -0.02196272648870945, -0.014551658183336258, -0.025384260341525078, -0.01898748055100441, 0.008722880855202675, -0.005027489736676216, -0.009831836447119713, -0.004131535068154335, 0.0011427650460973382, -0.0033674377482384443, -0.0006871804362162948, 0.20805084705352783, 0.008797261863946915, 0.009574883617460728, 0.04227554425597191, -0.0030293415766209364, -0.006325779017060995, 0.02783207595348358, 0.020867295563220978, 0.0016084924573078752, 0.022828252986073494, 0.0014039443340152502, 0.003218675497919321, -0.01704004593193531, 0.0015112898545339704, 0.004398630931973457, -0.0022872204426676035, -0.02904922142624855, -0.0007898771436884999, -0.018081381916999817, 0.0012873010709881783, 0.0004145904094912112, -0.004371583461761475, -0.023166349157691002, -0.007837069220840931, 0.02956312708556652, 0.006717970594763756, -0.029914747923612595, -0.0012670153519138694, 0.007140590809285641, -0.00031231631874106824, -0.013104607351124287, -0.010034694336354733, 0.0030817463994026184, -0.024653971195220947, 0.0036987720523029566, -0.009074500761926174, -0.011373554356396198, 0.0002628697548061609, 0.019001003354787827, 0.020393960177898407, -0.007945260033011436, -0.013929561711847782, 0.0012839201372116804, -0.020772628486156464, -0.001153753139078617, 0.025384260341525078, -0.0024748637806624174, -0.00691406661644578, 0.008675547316670418, 0.01663433015346527, -0.042654212564229965, 0.008398308418691158, 0.026493214070796967, 0.03743400797247887, -0.0032998186070472, 0.008364498615264893, 0.032889995723962784, 0.0019778625573962927, -0.0012560272589325905, 0.017783857882022858, -0.004520345479249954, 0.006271683610975742, -0.027074741199612617, 0.023342158645391464, -0.013591465540230274, -0.0024664115626364946, -0.014186514541506767, 0.015728233382105827, 0.007458401378244162, -0.00933145359158516, -0.006241254974156618, -0.010460695251822472, -0.0093855494633317, -0.012766511179506779, -0.012894987128674984, -0.022895872592926025, 0.012658320367336273, 0.028859887272119522, 0.014037752524018288, 0.042654212564229965, -0.010656790807843208, -0.0007801568717695773, -0.006978304591029882, -0.0019626482389867306, -0.017932619899511337, -0.02831893414258957, 0.01461927779018879, 0.011698126792907715, 0.014659848995506763, 0.002006600610911846, -0.022868823260068893, -0.017648618668317795, 0.0005616622511297464, -0.009047453291714191, 0.010453932918608189, 0.002789293183013797, 0.008662023581564426, 0.026533786207437515, -0.0036142480093985796, 0.021381201222538948, -0.002677721669897437, 0.0023734350688755512, 0.020921390503644943, 0.006538779474794865, -0.002762245712801814, 0.0012839201372116804, -0.003587200306355953, 0.011900984682142735, -0.0059606353752315044, 0.010163170285522938, -0.017973192036151886, -0.01986652985215187, 0.002045481698587537, 0.006308874115347862, 0.027777981013059616, 0.01828424073755741, 0.010068503208458424, -0.024369971826672554, -0.0029008649289608, -0.0004640369734261185, 0.00846592802554369, -0.031781040132045746, -0.007289353292435408, -0.012874701991677284, -0.021070152521133423, -0.004009820520877838, -0.010589171200990677, 0.018825193867087364, -0.010426885448396206, -0.0373799093067646, 0.008695833384990692, -0.016512615606188774, 0.021354153752326965, -0.003813724732026458, -0.0071946862153708935, 0.008472689427435398, -0.0019322194857522845, -0.02016405574977398, -0.0014774801675230265, -0.004189011175185442, -0.004118011333048344, -0.008520022965967655, 0.007742402143776417, 0.012475748546421528, 0.009514025412499905, -0.04971366003155708, 0.01468689739704132, 0.011779270134866238, -0.012455462478101254, -0.012583939358592033, -0.01821662113070488, 0.0021131010726094246, -0.005676634609699249, 0.0002041255502263084, 0.0271017886698246, -0.01898748055100441, -0.009081263095140457, -0.003590581240132451, -0.0005312336143106222, -0.0021350772585719824, -0.026669025421142578, 0.004608250688761473, 0.010974600911140442, -0.015430708415806293, -0.011116601526737213, -0.004571060184389353, -0.1764591485261917, 0.03824543580412865, 0.020772628486156464, -0.025357211008667946, 0.009196215309202671, 0.010284884832799435, 0.028075505048036575, -0.017824430018663406, 0.0021164820063859224, -0.013084321282804012, 0.014876230619847775, -0.0012991344556212425, -0.023125777021050453, -0.025478925555944443, -0.006224350072443485, 0.0029008649289608, 0.00325417541898787, -0.00437834532931447, 0.009094786830246449, 0.0065827323123812675, 0.04197802022099495, -0.020096436142921448, 0.012191747315227985, -0.016756044700741768, -0.002899174578487873, 0.019704243168234825, -0.005744253750890493, -0.015931090340018272, -0.023531492799520493, -0.022909395396709442, -0.032700661569833755, -0.035675905644893646, 0.01946081407368183, 0.0077491640113294125, -0.0008350975112989545, 0.0006309719756245613, 0.022043868899345398, -0.019109195098280907, -0.018906336277723312, 0.02603340335190296, -0.008161641657352448, 0.05041689798235893, 0.023450350388884544, -0.006153350230306387, -0.004919298924505711, 0.007526020519435406, -0.009635740891098976, 0.0012433485826477408, 0.010271361097693443, -0.015525375492870808, 0.02082672342658043, 0.008939262479543686, 0.009987360797822475, 0.013706417754292488, 0.023680254817008972, 0.0072217341512441635, 0.0025898164603859186, 0.01887928880751133, -0.011549364775419235, -0.015024993568658829, -0.016296233981847763, -0.031456466764211655, -0.01010231301188469, -0.017053570598363876, -0.008073735982179642, -0.03078027442097664, -0.043871358036994934, 0.002796055283397436, -0.030428653582930565, -0.013266893103718758, -0.0069512571208179, -0.006200683303177357, 0.025384260341525078, -0.012198509648442268, -0.012610986828804016, 0.02031281776726246, -0.015674138441681862, 0.012610986828804016, -0.0030597702134400606, -0.0027453408110886812, -0.01718880794942379, 0.04000353813171387, -0.021759869530797005, -0.015119659714400768, 0.006606399081647396, 0.02048862725496292, 0.0002337089681532234, 0.006264921743422747, 0.0011774199083447456, -0.00043445357005111873, 0.021367676556110382, -0.023815494030714035, 0.002855221973732114, -0.015133184380829334, 0.005358824040740728, 0.020853770896792412, 0.024924449622631073, 0.01002793200314045, -0.007918211631476879, 0.00011706579243764281, 0.0174051895737648, 0.007627449464052916, -0.03007703460752964, 0.023774921894073486, 0.024653971195220947, -0.004016582388430834, 0.019163290038704872, 0.013821370899677277, 0.014200038276612759, 0.00335560436360538, -0.020042339339852333, 0.03380961716175079, 0.021638154983520508, 0.007181162480264902, 0.0023565301671624184, 0.004520345479249954, -0.012610986828804016, 0.003759629325941205, -0.005318252369761467, -0.009872407652437687, 0.0516340434551239, 0.011488507501780987, 0.007958783768117428, 0.004145058803260326, -0.020502150058746338, -0.0225848238915205, -0.11035458743572235, -0.02140824869275093, -0.021678725257515907, 0.024762162938714027, -0.006653732154518366, 0.011758984066545963, 0.014159467071294785, 0.01879814639687538, -0.003759629325941205, 0.021164819598197937, -0.013517084531486034, -0.015511851757764816, -0.02600635588169098, 0.002075910335406661, -0.005808492191135883, -0.002718293108046055, -0.017053570598363876, -0.004750250838696957, -0.024667495861649513, 0.017946144565939903, 0.021529963240027428, 0.008073735982179642, 0.010920505970716476, -0.018892813473939896, -0.014511086978018284, -0.020732056349515915, -0.03451285511255264, 0.01836538314819336, 0.02048862725496292, 0.005429824348539114, 0.026506738737225533, -0.010379551909863949, 0.02203034609556198, -0.009608692489564419, -0.008864881470799446, 0.004080820828676224, -0.008716118521988392, -0.026669025421142578, 0.03962486982345581, -0.010284884832799435, 0.016539664939045906, 0.011684603057801723, 0.010007645934820175, -0.02335568331182003, 0.0013439322356134653, -0.015958137810230255, 0.0030851273331791162, 0.02831893414258957, 0.022922920063138008, -0.02761569432914257, -0.02956312708556652, -0.013388607650995255, -0.018757574260234833, -0.014903279021382332, 0.04690070077776909, -0.008479451760649681, 0.007397544104605913, 0.042140305042266846, -0.010751457884907722, 0.00762068759649992, -0.003925296477973461, -0.01044717151671648, 0.00465558422729373, 0.03375551849603653, 0.005013966001570225, 0.009202977642416954, -0.03367437794804573, -0.0258575938642025, -0.00462853629142046, -0.008256307803094387, 0.0025036020670086145, 0.009169167838990688, -0.008459165692329407, 0.016728997230529785, -0.021719297394156456, 0.0016879450995475054, -0.017094140872359276, -0.03537838160991669, 0.02545187808573246, -0.02111072465777397, -0.00931116845458746, -0.018054334446787834, 0.006122921593487263, 0.0012678606435656548, 0.03324161469936371, 0.01777033321559429, -0.002527268836274743, 0.027967313304543495, -0.006258159875869751, -0.026385024189949036, -0.005926825571805239, 0.0028721268754452467, 0.007979068905115128, -0.019487863406538963, -0.0027402692940086126, -0.002130005741491914, -0.0010903601069003344, -0.013611751608550549, -0.0006373112555593252, 0.006055301986634731, -0.00042198627488687634, -0.010913743637502193, -0.06550951302051544, 0.01357117947191, -0.020069388672709465, -0.011420887894928455, 0.025911688804626465, 0.0040503921918570995, 0.008479451760649681, -0.024694543331861496, -0.006413684226572514, 0.0003294324560556561, -0.014700421132147312, 0.0005565907922573388, -0.013280416838824749, -0.0034519617911428213, -0.011332983151078224, -0.018865766003727913, 0.022814728319644928, 0.002008291194215417, -0.0005092573119327426, 0.01571470871567726, 0.007086495403200388, 0.01386194210499525, 0.012110603973269463, -0.0010852887062355876, -0.009892693720757961, -0.004956489894539118, -0.02438349463045597, 0.013726703822612762, -0.00811430811882019, -0.007167638745158911, -0.0032676993869245052, -0.030104082077741623, 0.005524491425603628, 0.0450885035097599, -0.00014675485726911575, -0.018906336277723312, 0.008520022965967655, 0.007404305972158909, 0.0032795327715575695, 0.006670637056231499, -0.0050207278691232204, -0.04146411269903183, 0.017242904752492905, -0.01781090535223484, -0.006745018530637026, -0.009412596933543682, -0.031240085139870644, -0.007755925878882408, 0.008513261564075947, -0.020082911476492882, 0.02551949769258499, 0.00518639525398612, -0.01426765788346529, -0.0022906013764441013, -0.00861469004303217, -0.025938736274838448, 0.011150411330163479, 0.0012763129780068994, -0.003402937902137637, -0.004374964162707329, -0.0005937813548371196, 0.018460050225257874, 0.014565182849764824, -0.004371583461761475, 0.01847357489168644, 0.011948318220674992, 0.005003822967410088, -0.011245078407227993, 0.004145058803260326, -0.014159467071294785, -0.0335661880671978, -0.00039451595512218773, 0.022936442866921425, -0.0022534108720719814, 0.0015983495395630598, 0.006829542573541403, -0.005774682387709618, -0.009757455438375473, -0.013577941805124283, 0.029941795393824577, 0.007127067074179649, -0.0008591868681833148, -0.013821370899677277, 0.012171461246907711, 0.01766214333474636, -0.010906982235610485, -0.0070256381295621395, 0.002763936063274741, 0.004658964928239584, 0.008912215009331703, -0.03110484592616558, -0.003560152603313327, 0.0027081503067165613, -0.00335560436360538, 0.004189011175185442, 0.010785267688333988, -0.021016057580709457, -0.019041575491428375, 0.023896636441349983, 0.03816429525613785, 0.020393960177898407, -0.01766214333474636, -0.001857838360592723, -0.009317929856479168, -0.009148881770670414, 0.007965545170009136, -0.02097548544406891, -0.013841656967997551, -0.014795088209211826, 0.014105372130870819, 0.026439119130373, -0.014822135679423809, 0.012597463093698025, 0.02151643857359886, -0.013104607351124287, 0.023883111774921417, -0.0065996372140944, -0.03521609678864479, -0.017621571198105812, 0.01887928880751133, 0.031185990199446678, 0.021827487275004387, 0.01129241194576025, 0.005663110408931971, 0.01722938008606434, -0.00519653782248497, 0.01737814210355282, -0.03272770717740059, -0.0006571743870154023, 0.008357737213373184, -0.007208209950476885, -0.01858176477253437, -0.0425189733505249, 0.002968484302982688, -0.003763010259717703, -0.0075530679896473885, 0.016188044100999832, 0.02570883184671402, -0.035351336002349854, 0.041004300117492676, 0.005193157121539116, -0.0031983896624296904, -0.006592874880880117, 0.005798349156975746, 0.02269301377236843, 0.017472809180617332, 0.02125948667526245, -0.03857000917196274, -0.005673253443092108, 0.0021857917308807373, -0.011623745784163475, 0.00370891485363245, -0.011245078407227993, -0.01391603797674179, -0.014186514541506767, -0.030131129547953606, 0.0072149718180298805, -0.007336686830967665, 0.0064711603336036205, 0.016701949760317802, 0.009831836447119713, 0.0070459237322211266, -0.003925296477973461, -0.013679370284080505, -0.022152060642838478, 0.02641207166016102, -0.01866290718317032, -0.010812315158545971, -0.004993680398911238, 0.015214326791465282, 0.00982507411390543, -0.04403364285826683, 0.0045981076546013355, 0.00671120872721076, -0.014308229088783264, -0.01016993261873722, -0.01616099663078785, 0.0003716944484040141, -0.005027489736676216, -0.009135358035564423, 0.016350330784916878, -0.02937379479408264, -0.008499737828969955, 0.02750750258564949, 0.031240085139870644, -0.006055301986634731, 0.0044932980090379715, -0.01913624256849289])
print(simsearch_by_vector)

[Document(metadata={'id': 8, 'summary': 'Nasty No flavor'}, page_content='The candy is just red , No flavor . Just  plan and chewy .  I would never buy them again'), Document(metadata={'id': 4, 'summary': 'stale product'}, page_content='Arrived in 6 days and were so stale i could not eat any of the 6 bags!!'), Document(metadata={'id': 3, 'summary': 'Taste is neutral, quantity is DECEITFUL!'}, page_content='The taste of these white cheddar flat breads is like a regular cracker - which is not bad, except that I bought them because I wanted a cheese taste.<br /><br />What was a HUGE disappointment? How misleading the packaging of the box is. The photo on the box (I bought these in store) makes it look like it is full of long flatbreads (expanding the length and width of the box). Wrong! The plastic tray that holds the crackers is about 2 smaller all around - leaving you with about 15 or so small flatbreads.<br /><br />What is also bad about this is that the company states they use biodegr

In [34]:
# Similarity Search with Score if you already have embeddings you want to search on 
simsearch_by_vector_with_score = vector_store.similarity_search_by_vector_with_score([-0.0033353185281157494, -0.017689190804958344, -0.01590404286980629, -0.01751338131725788, -0.018054334446787834, 0.021841011941432953, -0.012313461862504482, -0.02273358590900898, -0.021286534145474434, -0.01814900152385235, 0.012252604588866234, 0.038759343326091766, 0.0015408731997013092, -0.00691406661644578, -0.013638799078762531, 0.024153590202331543, 0.039895348250865936, 0.0012036223197355866, 0.009372025728225708, -0.012178223580121994, -0.019853007048368454, 0.006024873349815607, 0.011319459415972233, -0.025167878717184067, -0.00759363966062665, 0.010284884832799435, 0.009831836447119713, -0.008492975495755672, -0.005639444105327129, -0.009446406736969948, 0.007444877177476883, -0.009277358651161194, -0.025289593264460564, -0.02119186706840992, -0.005906539969146252, -0.018906336277723312, -0.007539544254541397, -0.016066329553723335, -0.01171841286122799, -0.02093491330742836, 0.004608250688761473, 0.011042220517992973, 0.011549364775419235, -0.009541073814034462, 0.0025864355266094208, 0.0026202453300356865, -0.0036007240414619446, -0.011995651759207249, -0.02549245022237301, -0.007958783768117428, 0.015701185911893845, 0.016188044100999832, -0.005825396627187729, -0.00866878591477871, -0.00038881058571860194, -0.0006356207886710763, 0.0074110678397119045, 0.00766802066937089, -0.005419681314378977, -0.007674783002585173, 0.0086823096498847, -0.004740108270198107, -0.01406479999423027, 0.02170577272772789, -0.0029955320060253143, -0.008574118837714195, 0.005460252985358238, 0.0034130807034671307, -0.005521110258996487, 0.0045507741160690784, 0.03662257641553879, -0.004141678102314472, 0.004442583303898573, -0.0035026762634515762, 0.0021316963247954845, -0.009297644719481468, -0.032186754047870636, 0.01478156354278326, -0.0016355401603505015, -0.005835539661347866, 0.03253837302327156, -0.040301062166690826, -0.0090339295566082, -0.001611873391084373, 0.018703479319810867, -0.00610601669177413, -0.016323283314704895, 0.002358220750465989, -0.004118011333048344, 0.005784825421869755, 0.01579585298895836, 0.028886936604976654, -0.004111249465495348, 0.020475102588534355, -0.0360545739531517, 0.0018696717452257872, 0.0039658681489527225, 0.026723120361566544, -0.011177458800375462, -0.03759629279375076, 0.0012501105666160583, 0.007478686980903149, -0.03445876017212868, -0.011130125261843204, -0.019677195698022842, -0.004784060642123222, 0.01866290718317032, 0.0013667537132278085, 0.015417184680700302, -0.01467337366193533, -0.010109075345098972, 0.03976010903716087, 0.02482978254556656, -0.045196693390607834, -0.02562768943607807, -0.0017614809330552816, -0.0002624471380840987, -0.006839685142040253, 0.005757777485996485, -0.001704004593193531, 0.020650913938879967, 0.021841011941432953, 0.045845840126276016, 0.00012234854511916637, -0.0019170051673427224, 0.006082349922508001, -0.027940265834331512, 0.005524491425603628, -0.002109719906002283, 0.011393840424716473, 0.036784861236810684, 0.019257957115769386, 0.0018020524876192212, 0.006051921285688877, -0.01858176477253437, 0.021584058180451393, -0.0070459237322211266, 0.00785735435783863, -0.00133378931786865, -0.028048457577824593, -0.006944495253264904, 0.019744815304875374, -0.025587117299437523, -0.020556246861815453, 0.00877697579562664, 0.03251132741570473, 0.01059593353420496, 0.00782354548573494, 0.009493740275502205, 0.008236022666096687, 0.002108029555529356, -0.007999354973435402, 0.012692130170762539, -0.02217910811305046, 0.0018848860636353493, 0.021178342401981354, 0.00394896324723959, 0.02273358590900898, -0.006907304283231497, 0.014754516072571278, 0.007830306887626648, 0.012090318836271763, -0.0025661499239504337, 0.0011884080013260245, -0.005483919754624367, 0.03640619292855263, 0.022192630916833878, 0.00766802066937089, -0.004368202295154333, -0.0065353987738490105, -0.002111410489305854, 0.02401835098862648, 0.0008110081544145942, 0.012225557118654251, -0.01879814639687538, 0.020258720964193344, -0.009148881770670414, 0.00009382168354932219, -0.0399494431912899, -0.009764216840267181, -0.009656026028096676, -0.0017800763016566634, 0.03110484592616558, 0.029617223888635635, 0.0030479368288069963, -0.0005232038092799485, 0.017242904752492905, -0.02500559203326702, 0.01663433015346527, -0.0012703962856903672, 0.0030428653117269278, 0.010237551294267178, 0.031023703515529633, -0.01101517304778099, -0.6837656497955322, 0.0020471722818911076, -0.00041289994260296226, 0.007877640426158905, 0.018973955884575844, 0.020096436142921448, 0.006109397392719984, 0.007999354973435402, -0.018162526190280914, 0.0011757294414564967, -0.007803259417414665, 0.019190337508916855, -0.009669549763202667, -0.0013912656577304006, -0.0061127785593271255, 0.006575970444828272, 0.0009407525649294257, -0.013997181318700314, 0.004821251146495342, 0.012894987128674984, 0.01016993261873722, -0.0009990741964429617, -0.0026692692190408707, -0.008648499846458435, -0.008154879324138165, -0.018892813473939896, -0.0012788487365469337, -0.0019186957506462932, 0.0045981076546013355, 0.01714823767542839, -0.007018876262009144, 0.01300317794084549, -0.011427650228142738, 0.001988005358725786, 0.03518904745578766, -0.0015366470906883478, 0.003017508191987872, 0.0399494431912899, 0.010508028790354729, 0.051796332001686096, -0.017351094633340836, -0.018189573660492897, 0.015701185911893845, 0.00895954854786396, -0.024410542100667953, 0.0016237067757174373, 0.008161641657352448, 0.016323283314704895, 0.010629743337631226, -0.004929441958665848, -0.01866290718317032, 0.0015729924198240042, 0.015782328322529793, 0.002850150689482689, 0.003749486291781068, 0.006153350230306387, 0.023301586508750916, -0.02357206493616104, 0.024369971826672554, 0.0009737169602885842, 0.016539664939045906, 0.011813079938292503, -0.015579471364617348, -0.034648094326257706, -0.01755395159125328, 0.005051156505942345, -0.03202446922659874, 0.0069512571208179, 0.006437350995838642, -0.013402131386101246, 0.014889754354953766, 0.032781802117824554, -0.010413361713290215, -0.006369731388986111, 0.006417064927518368, 0.02078615128993988, 0.001138538820669055, -0.008323927409946918, -0.0180678591132164, -0.010000884532928467, -0.008560595102608204, -0.012536605820059776, -0.039246201515197754, 0.003942201379686594, 0.006295350380241871, 0.005460252985358238, -0.01954195834696293, -0.000006484578534582397, 0.011393840424716473, -0.007640973199158907, 0.002662507351487875, 0.03786677122116089, -0.007952021434903145, 0.0021993154659867287, -0.0058118728920817375, 0.022706538438796997, 0.0061702546663582325, 0.011522317305207253, 0.011278888210654259, -0.04157230257987976, -0.010589171200990677, 0.0006880256696604192, 0.027277598157525063, 0.0007429663091897964, 0.024951497092843056, 0.020961962640285492, -0.003031032159924507, 0.01539013721048832, 0.008161641657352448, -0.00791145022958517, 0.007086495403200388, 0.004026725422590971, 0.0037765339948236942, -0.01173193659633398, 0.003877962939441204, -0.02658788114786148, -0.005189775954931974, 0.009710121899843216, 0.024356447160243988, -0.030861416831612587, -0.0016406115610152483, -0.012976130470633507, 0.01461927779018879, -0.01729699969291687, 0.004926060792058706, 0.011143648996949196, -0.026357976719737053, -0.032592467963695526, 0.011123363859951496, -0.028156647458672523, -0.0021131010726094246, -0.01645852066576481, 0.02391016110777855, -0.025478925555944443, 0.018040811643004417, 0.01766214333474636, 0.019636625424027443, 0.013713180087506771, 0.00518639525398612, -0.010359265841543674, -0.014240610413253307, 0.01369965635240078, -0.00023413158487528563, -0.006859971210360527, 0.004253249615430832, -0.00883107166737318, -0.004320868756622076, -0.016201568767428398, -0.0029093173798173666, 0.012097080238163471, -0.00818868912756443, 0.005000442266464233, 0.0029279126320034266, 0.00014886796998325735, 0.029833605512976646, -0.028670554980635643, -0.006518493872135878, -0.005686777178198099, 0.00618039770051837, 0.010251075029373169, 0.01714823767542839, 0.019298529252409935, -0.024437589570879936, -0.0022720061242580414, -0.012103842571377754, -0.03540543094277382, 0.007032399997115135, 0.03359323367476463, -0.009493740275502205, -0.03562181070446968, 0.01555242296308279, -0.002633769065141678, -0.031753990799188614, 0.009466692805290222, 0.022084441035985947, 0.011684603057801723, 0.0029076270293444395, -0.025789974257349968, -0.001874743145890534, 0.011934794485569, 0.006846447009593248, -0.012678605504333973, -0.011400602757930756, -0.012381081469357014, 0.014524610713124275, 0.010785267688333988, 0.03635209798812866, 0.03851591423153877, -0.031077798455953598, 0.011265363544225693, -0.014429943636059761, 0.006532017607241869, 0.01411889586597681, 0.007133828941732645, 0.003715676721185446, -0.020475102588534355, 0.008283356204628944, 0.013530608266592026, 0.026804262772202492, 0.01571470871567726, 0.017905572429299355, -0.008364498615264893, 0.012719177640974522, -0.013091083616018295, 0.018230143934488297, -0.039976488798856735, 0.0017969810869544744, -0.019677195698022842, 0.016728997230529785, 0.012103842571377754, -0.01590404286980629, -0.013503560796380043, 0.0016617425717413425, -0.005700301378965378, 0.009737169370055199, 0.0218815840780735, -0.005507586523890495, 0.010305170901119709, 0.010406599380075932, -0.00954783521592617, -0.000835942744743079, -0.009405835531651974, 0.0008165022009052336, 0.004270154517143965, -0.006227731239050627, 0.014551658183336258, -0.002627007197588682, 0.03667667135596275, -0.004817870445549488, -0.010420123115181923, -0.02159758284687996, 0.004067296627908945, -0.0032355801668018103, 0.011921270750463009, 0.003877962939441204, 0.008006117306649685, 0.014889754354953766, -0.022206155583262444, 0.03359323367476463, -0.007755925878882408, 0.006095873657613993, 0.03264656662940979, 0.022828252986073494, -0.01755395159125328, 0.012462224811315536, 0.006488065235316753, 0.024951497092843056, -0.007282591424882412, -0.007952021434903145, -0.008012878708541393, -0.012861178256571293, -0.009047453291714191, -0.017783857882022858, 0.013462988659739494, 0.015484804287552834, -0.019406719133257866, 0.0219221543520689, 0.004243106581270695, 0.010934029705822468, 0.022977015003561974, 0.008371260948479176, 0.013429179787635803, 0.0024748637806624174, -0.002767316997051239, 0.03148351237177849, 0.004814489278942347, 0.005051156505942345, -0.02996884286403656, 0.013456227257847786, 0.015119659714400768, -0.015876995399594307, -0.0003748641174752265, 0.00811430811882019, -0.011373554356396198, 0.015620042569935322, 0.02346387319266796, -0.01836538314819336, 0.02038043551146984, 0.0040503921918570995, -0.01129241194576025, -0.038948677480220795, -0.04092315956950188, 0.023964256048202515, 0.015065564773976803, 0.016688426956534386, 0.007640973199158907, -0.035757049918174744, -0.014727468602359295, -0.001906862366013229, 0.020299293100833893, -0.0009948479710146785, 0.019636625424027443, 0.000619138590991497, 0.008946023881435394, -0.0012264437973499298, -0.004172106739133596, 0.03664962202310562, -0.006991828326135874, -0.000013986086742079351, -0.010203742422163486, 0.015430708415806293, -0.007958783768117428, -0.017026523128151894, -0.02684483490884304, -0.006481303367763758, 0.008837834000587463, -0.020610341802239418, -0.020921390503644943, -0.03018522448837757, 0.004178868606686592, -0.01785147748887539, 0.007891164161264896, -0.01533604133874178, -0.006373112555593252, -0.004882108420133591, 0.013523845933377743, -0.007323162630200386, -0.011062506586313248, 0.02564121223986149, -0.00813459325581789, -0.02251720428466797, -0.012482509948313236, -0.003969248849898577, -0.014281181618571281, 0.08265774697065353, 0.036162763833999634, -0.0014800159260630608, 0.029481984674930573, 0.021016057580709457, -0.011346506886184216, -0.020989010110497475, 0.0023193396627902985, 0.020137006416916847, -0.004405392799526453, 0.0056428248062729836, 0.0005396860069595277, 0.011332983151078224, -0.015376613475382328, 0.01555242296308279, 0.007444877177476883, -0.0005599717842414975, -0.010643267072737217, 0.016282711178064346, -0.008432118222117424, -0.004780679475516081, -0.018960433080792427, 0.0024004827719181776, 0.005761158652603626, -0.004124773200601339, -0.0139025142416358, 0.00620744563639164, 0.023599112406373024, -0.008973072282969952, -0.008601166307926178, -0.009412596933543682, 0.03540543094277382, -0.0007729723583906889, -0.0038982487749308348, 0.0037190576549619436, 0.006075588054955006, -0.00529796676710248, 0.015579471364617348, 0.04195097088813782, 0.0069208284839987755, 0.01421356201171875, 0.02508673444390297, -0.011447936296463013, -0.0036548194475471973, 0.021205391734838486, -0.025397783145308495, -0.008073735982179642, 0.03188923001289368, 0.0056800153106451035, -0.01307755894958973, 0.03416123613715172, 0.023233968764543533, -0.016404425725340843, -0.01954195834696293, 0.0044087739661335945, -0.007512496784329414, 0.01326013170182705, 0.003272770904004574, -0.0028907221276313066, -0.006433969829231501, -0.02823779173195362, -0.021732820197939873, 0.0006643589586019516, -0.007789735682308674, 0.005456871818751097, -0.04333040490746498, -0.023477397859096527, -0.004290440119802952, -0.020069388672709465, -0.006538779474794865, -0.024910924956202507, -0.011258602142333984, -0.010095551609992981, 0.017581000924110413, 0.015024993568658829, 0.010129360482096672, 0.028589410707354546, -0.0029228413477540016, 0.005855825264006853, 0.02112424746155739, -0.003303199540823698, -0.016512615606188774, 0.013807847164571285, -0.005889635067433119, -0.004905775189399719, 0.020907865837216377, -0.0023700541350990534, 0.009919741190969944, -0.006741637364029884, 0.005737491883337498, 0.02067796140909195, 0.0020184339955449104, 0.02280120551586151, 0.0014926944859325886, -0.0048787277191877365, 0.005345300305634737, 0.015876995399594307, 0.014254134148359299, -0.007701830472797155, 0.0032000800129026175, 0.02449168637394905, -0.02035338804125786, -0.032998185604810715, 0.002412316156551242, 0.0035533905029296875, 0.008296879939734936, 0.004834774881601334, -0.005629301071166992, 0.003272770904004574, 0.00027660492924042046, 0.017094140872359276, -0.014497563242912292, 0.015227850526571274, -0.004178868606686592, 0.014362324960529804, 0.012381081469357014, -0.008526785299181938, 0.017269952222704887, 0.002092815237119794, -0.02309872955083847, -0.024802733212709427, -0.015322517603635788, 0.042951736599206924, 0.032186754047870636, -0.01854119263589382, -0.005328395403921604, 0.0031764134764671326, 0.010136122815310955, 0.004358059260994196, 0.01461927779018879, 0.005598872434347868, 0.028156647458672523, -0.002091124653816223, -0.02111072465777397, -0.0181084293872118, -0.017026523128151894, 0.0034299856051802635, 0.016661379486322403, -0.01244870014488697, -0.0023886493872851133, -0.017635095864534378, -0.007864116691052914, 0.007282591424882412, -0.019677195698022842, -0.011576412245631218, -0.04995708912611008, -0.026357976719737053, -0.012807082384824753, 0.015701185911893845, 0.011725175194442272, -0.014240610413253307, -0.00020021632371935993, -0.009635740891098976, 0.01571470871567726, -0.0053351572714746, -0.033322758972644806, 0.0010252766078338027, 0.007140590809285641, 0.0012873010709881783, 0.04278944805264473, 0.024924449622631073, -0.00438848789781332, 0.01327365543693304, 0.020961962640285492, -0.01075821928679943, 0.004124773200601339, -0.00674839923158288, 0.005700301378965378, -0.019596053287386894, 0.010609457269310951, 0.015444232150912285, -0.00918269157409668, 0.023058157414197922, -0.0013380155432969332, -0.042032115161418915, 0.00910831056535244, 0.010906982235610485, -0.009757455438375473, -0.006616541650146246, -0.0024731734301894903, -0.004209297243505716, -0.003472247626632452, 0.014132419601082802, 0.01708061806857586, 0.005338538438081741, -0.00039451595512218773, -0.00031675383797846735, 0.020191103219985962, 0.006829542573541403, -0.0036852480843663216, 0.021381201222538948, -0.013665846548974514, 0.006126302294433117, -0.008662023581564426, -0.009419359266757965, -0.006427207961678505, -0.013692894019186497, 0.005710443947464228, 0.0032609375193715096, 0.010548599995672703, 0.02093491330742836, 0.022422537207603455, -0.0051728710532188416, 0.016147471964359283, -0.009906217455863953, -0.0016152544412761927, -0.015011469833552837, -0.0263985488563776, 0.012922035530209541, -0.028589410707354546, -0.01851414516568184, -0.027480455115437508, -0.027967313304543495, -0.02523549646139145, 0.000527007388882339, 0.008107545785605907, -0.005051156505942345, 0.0006026563933119178, 0.006707827560603619, -0.0032507944852113724, 0.0044662500731647015, 0.0012188366381451488, 0.005740872584283352, -0.007938497699797153, 0.028913984075188637, 0.03819134086370468, -0.007377258036285639, -0.015890520066022873, 0.023490920662879944, 0.012529843486845493, 0.005003822967410088, 0.012130890041589737, 0.0027537932619452477, 0.003877962939441204, -0.01795966736972332, 0.02196272648870945, 0.004111249465495348, -0.006562446244060993, -0.0453319326043129, 0.03743400797247887, 0.011894222348928452, -0.012070032767951488, -0.0011410745792090893, -0.015322517603635788, -0.03716352954506874, 0.025573592633008957, -0.024194160476326942, -0.010142885148525238, -0.03797496110200882, -0.0016237067757174373, -0.0206373892724514, 0.006461017765104771, -0.01582290045917034, 0.034215331077575684, 0.0005451800534501672, -0.004834774881601334, 0.0035128190647810698, -0.0020116721279919147, -0.002542483154684305, -0.02487035281956196, -0.00684982817620039, 0.034945618361234665, -0.018243668600916862, -0.005220204591751099, 0.01357117947191, 0.000029187207474024035, -0.015417184680700302, 0.009088024497032166, -0.02078615128993988, 0.029022173956036568, -0.025073211640119553, -0.013449464924633503, -0.00032731934334151447, -0.009419359266757965, 0.015268422663211823, 0.003908391576260328, -0.013483274728059769, -0.013158702291548252, -0.007728877943009138, -0.025898166000843048, 0.005108633078634739, 0.0037765339948236942, 0.0006935197161510587, -0.0006271683960221708, -0.02119186706840992, -0.0033505328465253115, -0.01571470871567726, -0.006802494637668133, -0.0036345336120575666, 0.012536605820059776, -0.003712295787408948, -0.011008410714566708, 0.007086495403200388, 0.002074219984933734, 0.005176252219825983, -0.0018121954053640366, 0.006038397550582886, 0.02031281776726246, -0.02812959998846054, 0.01659375987946987, -0.020989010110497475, 0.004327630624175072, -0.03951667994260788, -0.0003068222722504288, 0.008506499230861664, -0.0016177900834009051, -0.002733507426455617, 0.007546306122094393, -0.004611631389707327, 0.005135680548846722, -0.006897161714732647, 0.017567476257681847, 0.0023227205965667963, -0.013050511479377747, 0.01582290045917034, 0.003830629400908947, 0.010453932918608189, -0.03562181070446968, -0.034837428480386734, 0.02802141010761261, -0.006484684068709612, 0.016363853588700294, 0.0020319579634815454, -0.019961196929216385, -0.007627449464052916, -0.00048094178782776, 0.031862180680036545, 0.014943850226700306, -0.015457755886018276, -0.0232069194316864, -0.006396779324859381, 0.00875669065862894, -0.029481984674930573, -0.01468689739704132, -0.029941795393824577, -0.02523549646139145, -0.007877640426158905, 0.02530311606824398, 0.023585587739944458, -0.0030006032902747393, 0.02211148850619793, 0.016553187742829323, 0.0005071442574262619, -0.0021688868291676044, -0.021570535376667976, -0.035757049918174744, -0.008039926178753376, -0.012766511179506779, -0.016093377023935318, -0.009027167223393917, -0.016661379486322403, 0.02277415804564953, -0.0002588548813946545, -0.020515674725174904, -0.0070526860654354095, -0.004861822817474604, -0.01744576171040535, -0.002293982543051243, 0.00791145022958517, 0.025032639503479004, 0.013307464309036732, 0.00987916998565197, -0.006362969521433115, 0.016255663707852364, -0.000155946851009503, 0.04208621010184288, -0.0213406290858984, -0.023477397859096527, 0.01384841836988926, -0.02228729799389839, 0.0022212916519492865, -0.013983656652271748, 0.002599959494546056, -0.03026636876165867, 0.009074500761926174, 0.01630975864827633, 0.004425678867846727, 0.02641207166016102, 0.00650835083797574, -0.013395369984209538, -0.021638154983520508, 0.01946081407368183, 0.0038002007640898228, 0.0021688868291676044, 0.013402131386101246, -0.008858119137585163, -0.0139025142416358, 0.005115394946187735, -0.02934674732387066, -0.014091847464442253, 0.0019170051673427224, -0.0014808612177148461, -0.000039699883927823976, 0.01498442143201828, -0.01243517640978098, -0.013375083915889263, -0.02170577272772789, -0.03151056170463562, -0.010609457269310951, -0.002765626646578312, -0.013780799694359303, 0.030942561104893684, -0.002380196936428547, -0.014605754055082798, -0.01943376660346985, 0.0004572750476654619, -0.011272125877439976, -0.010501266457140446, 0.003881343873217702, -0.00335560436360538, 0.004909156356006861, -0.019190337508916855, 0.017107665538787842, -0.03786677122116089, 0.010832601226866245, 0.002527268836274743, 0.003763010259717703, 0.015511851757764816, 0.015890520066022873, 0.008912215009331703, -0.032403137534856796, -0.011738698929548264, -0.012590700760483742, -0.009845360182225704, -0.008161641657352448, -0.02196272648870945, -0.014551658183336258, -0.025384260341525078, -0.01898748055100441, 0.008722880855202675, -0.005027489736676216, -0.009831836447119713, -0.004131535068154335, 0.0011427650460973382, -0.0033674377482384443, -0.0006871804362162948, 0.20805084705352783, 0.008797261863946915, 0.009574883617460728, 0.04227554425597191, -0.0030293415766209364, -0.006325779017060995, 0.02783207595348358, 0.020867295563220978, 0.0016084924573078752, 0.022828252986073494, 0.0014039443340152502, 0.003218675497919321, -0.01704004593193531, 0.0015112898545339704, 0.004398630931973457, -0.0022872204426676035, -0.02904922142624855, -0.0007898771436884999, -0.018081381916999817, 0.0012873010709881783, 0.0004145904094912112, -0.004371583461761475, -0.023166349157691002, -0.007837069220840931, 0.02956312708556652, 0.006717970594763756, -0.029914747923612595, -0.0012670153519138694, 0.007140590809285641, -0.00031231631874106824, -0.013104607351124287, -0.010034694336354733, 0.0030817463994026184, -0.024653971195220947, 0.0036987720523029566, -0.009074500761926174, -0.011373554356396198, 0.0002628697548061609, 0.019001003354787827, 0.020393960177898407, -0.007945260033011436, -0.013929561711847782, 0.0012839201372116804, -0.020772628486156464, -0.001153753139078617, 0.025384260341525078, -0.0024748637806624174, -0.00691406661644578, 0.008675547316670418, 0.01663433015346527, -0.042654212564229965, 0.008398308418691158, 0.026493214070796967, 0.03743400797247887, -0.0032998186070472, 0.008364498615264893, 0.032889995723962784, 0.0019778625573962927, -0.0012560272589325905, 0.017783857882022858, -0.004520345479249954, 0.006271683610975742, -0.027074741199612617, 0.023342158645391464, -0.013591465540230274, -0.0024664115626364946, -0.014186514541506767, 0.015728233382105827, 0.007458401378244162, -0.00933145359158516, -0.006241254974156618, -0.010460695251822472, -0.0093855494633317, -0.012766511179506779, -0.012894987128674984, -0.022895872592926025, 0.012658320367336273, 0.028859887272119522, 0.014037752524018288, 0.042654212564229965, -0.010656790807843208, -0.0007801568717695773, -0.006978304591029882, -0.0019626482389867306, -0.017932619899511337, -0.02831893414258957, 0.01461927779018879, 0.011698126792907715, 0.014659848995506763, 0.002006600610911846, -0.022868823260068893, -0.017648618668317795, 0.0005616622511297464, -0.009047453291714191, 0.010453932918608189, 0.002789293183013797, 0.008662023581564426, 0.026533786207437515, -0.0036142480093985796, 0.021381201222538948, -0.002677721669897437, 0.0023734350688755512, 0.020921390503644943, 0.006538779474794865, -0.002762245712801814, 0.0012839201372116804, -0.003587200306355953, 0.011900984682142735, -0.0059606353752315044, 0.010163170285522938, -0.017973192036151886, -0.01986652985215187, 0.002045481698587537, 0.006308874115347862, 0.027777981013059616, 0.01828424073755741, 0.010068503208458424, -0.024369971826672554, -0.0029008649289608, -0.0004640369734261185, 0.00846592802554369, -0.031781040132045746, -0.007289353292435408, -0.012874701991677284, -0.021070152521133423, -0.004009820520877838, -0.010589171200990677, 0.018825193867087364, -0.010426885448396206, -0.0373799093067646, 0.008695833384990692, -0.016512615606188774, 0.021354153752326965, -0.003813724732026458, -0.0071946862153708935, 0.008472689427435398, -0.0019322194857522845, -0.02016405574977398, -0.0014774801675230265, -0.004189011175185442, -0.004118011333048344, -0.008520022965967655, 0.007742402143776417, 0.012475748546421528, 0.009514025412499905, -0.04971366003155708, 0.01468689739704132, 0.011779270134866238, -0.012455462478101254, -0.012583939358592033, -0.01821662113070488, 0.0021131010726094246, -0.005676634609699249, 0.0002041255502263084, 0.0271017886698246, -0.01898748055100441, -0.009081263095140457, -0.003590581240132451, -0.0005312336143106222, -0.0021350772585719824, -0.026669025421142578, 0.004608250688761473, 0.010974600911140442, -0.015430708415806293, -0.011116601526737213, -0.004571060184389353, -0.1764591485261917, 0.03824543580412865, 0.020772628486156464, -0.025357211008667946, 0.009196215309202671, 0.010284884832799435, 0.028075505048036575, -0.017824430018663406, 0.0021164820063859224, -0.013084321282804012, 0.014876230619847775, -0.0012991344556212425, -0.023125777021050453, -0.025478925555944443, -0.006224350072443485, 0.0029008649289608, 0.00325417541898787, -0.00437834532931447, 0.009094786830246449, 0.0065827323123812675, 0.04197802022099495, -0.020096436142921448, 0.012191747315227985, -0.016756044700741768, -0.002899174578487873, 0.019704243168234825, -0.005744253750890493, -0.015931090340018272, -0.023531492799520493, -0.022909395396709442, -0.032700661569833755, -0.035675905644893646, 0.01946081407368183, 0.0077491640113294125, -0.0008350975112989545, 0.0006309719756245613, 0.022043868899345398, -0.019109195098280907, -0.018906336277723312, 0.02603340335190296, -0.008161641657352448, 0.05041689798235893, 0.023450350388884544, -0.006153350230306387, -0.004919298924505711, 0.007526020519435406, -0.009635740891098976, 0.0012433485826477408, 0.010271361097693443, -0.015525375492870808, 0.02082672342658043, 0.008939262479543686, 0.009987360797822475, 0.013706417754292488, 0.023680254817008972, 0.0072217341512441635, 0.0025898164603859186, 0.01887928880751133, -0.011549364775419235, -0.015024993568658829, -0.016296233981847763, -0.031456466764211655, -0.01010231301188469, -0.017053570598363876, -0.008073735982179642, -0.03078027442097664, -0.043871358036994934, 0.002796055283397436, -0.030428653582930565, -0.013266893103718758, -0.0069512571208179, -0.006200683303177357, 0.025384260341525078, -0.012198509648442268, -0.012610986828804016, 0.02031281776726246, -0.015674138441681862, 0.012610986828804016, -0.0030597702134400606, -0.0027453408110886812, -0.01718880794942379, 0.04000353813171387, -0.021759869530797005, -0.015119659714400768, 0.006606399081647396, 0.02048862725496292, 0.0002337089681532234, 0.006264921743422747, 0.0011774199083447456, -0.00043445357005111873, 0.021367676556110382, -0.023815494030714035, 0.002855221973732114, -0.015133184380829334, 0.005358824040740728, 0.020853770896792412, 0.024924449622631073, 0.01002793200314045, -0.007918211631476879, 0.00011706579243764281, 0.0174051895737648, 0.007627449464052916, -0.03007703460752964, 0.023774921894073486, 0.024653971195220947, -0.004016582388430834, 0.019163290038704872, 0.013821370899677277, 0.014200038276612759, 0.00335560436360538, -0.020042339339852333, 0.03380961716175079, 0.021638154983520508, 0.007181162480264902, 0.0023565301671624184, 0.004520345479249954, -0.012610986828804016, 0.003759629325941205, -0.005318252369761467, -0.009872407652437687, 0.0516340434551239, 0.011488507501780987, 0.007958783768117428, 0.004145058803260326, -0.020502150058746338, -0.0225848238915205, -0.11035458743572235, -0.02140824869275093, -0.021678725257515907, 0.024762162938714027, -0.006653732154518366, 0.011758984066545963, 0.014159467071294785, 0.01879814639687538, -0.003759629325941205, 0.021164819598197937, -0.013517084531486034, -0.015511851757764816, -0.02600635588169098, 0.002075910335406661, -0.005808492191135883, -0.002718293108046055, -0.017053570598363876, -0.004750250838696957, -0.024667495861649513, 0.017946144565939903, 0.021529963240027428, 0.008073735982179642, 0.010920505970716476, -0.018892813473939896, -0.014511086978018284, -0.020732056349515915, -0.03451285511255264, 0.01836538314819336, 0.02048862725496292, 0.005429824348539114, 0.026506738737225533, -0.010379551909863949, 0.02203034609556198, -0.009608692489564419, -0.008864881470799446, 0.004080820828676224, -0.008716118521988392, -0.026669025421142578, 0.03962486982345581, -0.010284884832799435, 0.016539664939045906, 0.011684603057801723, 0.010007645934820175, -0.02335568331182003, 0.0013439322356134653, -0.015958137810230255, 0.0030851273331791162, 0.02831893414258957, 0.022922920063138008, -0.02761569432914257, -0.02956312708556652, -0.013388607650995255, -0.018757574260234833, -0.014903279021382332, 0.04690070077776909, -0.008479451760649681, 0.007397544104605913, 0.042140305042266846, -0.010751457884907722, 0.00762068759649992, -0.003925296477973461, -0.01044717151671648, 0.00465558422729373, 0.03375551849603653, 0.005013966001570225, 0.009202977642416954, -0.03367437794804573, -0.0258575938642025, -0.00462853629142046, -0.008256307803094387, 0.0025036020670086145, 0.009169167838990688, -0.008459165692329407, 0.016728997230529785, -0.021719297394156456, 0.0016879450995475054, -0.017094140872359276, -0.03537838160991669, 0.02545187808573246, -0.02111072465777397, -0.00931116845458746, -0.018054334446787834, 0.006122921593487263, 0.0012678606435656548, 0.03324161469936371, 0.01777033321559429, -0.002527268836274743, 0.027967313304543495, -0.006258159875869751, -0.026385024189949036, -0.005926825571805239, 0.0028721268754452467, 0.007979068905115128, -0.019487863406538963, -0.0027402692940086126, -0.002130005741491914, -0.0010903601069003344, -0.013611751608550549, -0.0006373112555593252, 0.006055301986634731, -0.00042198627488687634, -0.010913743637502193, -0.06550951302051544, 0.01357117947191, -0.020069388672709465, -0.011420887894928455, 0.025911688804626465, 0.0040503921918570995, 0.008479451760649681, -0.024694543331861496, -0.006413684226572514, 0.0003294324560556561, -0.014700421132147312, 0.0005565907922573388, -0.013280416838824749, -0.0034519617911428213, -0.011332983151078224, -0.018865766003727913, 0.022814728319644928, 0.002008291194215417, -0.0005092573119327426, 0.01571470871567726, 0.007086495403200388, 0.01386194210499525, 0.012110603973269463, -0.0010852887062355876, -0.009892693720757961, -0.004956489894539118, -0.02438349463045597, 0.013726703822612762, -0.00811430811882019, -0.007167638745158911, -0.0032676993869245052, -0.030104082077741623, 0.005524491425603628, 0.0450885035097599, -0.00014675485726911575, -0.018906336277723312, 0.008520022965967655, 0.007404305972158909, 0.0032795327715575695, 0.006670637056231499, -0.0050207278691232204, -0.04146411269903183, 0.017242904752492905, -0.01781090535223484, -0.006745018530637026, -0.009412596933543682, -0.031240085139870644, -0.007755925878882408, 0.008513261564075947, -0.020082911476492882, 0.02551949769258499, 0.00518639525398612, -0.01426765788346529, -0.0022906013764441013, -0.00861469004303217, -0.025938736274838448, 0.011150411330163479, 0.0012763129780068994, -0.003402937902137637, -0.004374964162707329, -0.0005937813548371196, 0.018460050225257874, 0.014565182849764824, -0.004371583461761475, 0.01847357489168644, 0.011948318220674992, 0.005003822967410088, -0.011245078407227993, 0.004145058803260326, -0.014159467071294785, -0.0335661880671978, -0.00039451595512218773, 0.022936442866921425, -0.0022534108720719814, 0.0015983495395630598, 0.006829542573541403, -0.005774682387709618, -0.009757455438375473, -0.013577941805124283, 0.029941795393824577, 0.007127067074179649, -0.0008591868681833148, -0.013821370899677277, 0.012171461246907711, 0.01766214333474636, -0.010906982235610485, -0.0070256381295621395, 0.002763936063274741, 0.004658964928239584, 0.008912215009331703, -0.03110484592616558, -0.003560152603313327, 0.0027081503067165613, -0.00335560436360538, 0.004189011175185442, 0.010785267688333988, -0.021016057580709457, -0.019041575491428375, 0.023896636441349983, 0.03816429525613785, 0.020393960177898407, -0.01766214333474636, -0.001857838360592723, -0.009317929856479168, -0.009148881770670414, 0.007965545170009136, -0.02097548544406891, -0.013841656967997551, -0.014795088209211826, 0.014105372130870819, 0.026439119130373, -0.014822135679423809, 0.012597463093698025, 0.02151643857359886, -0.013104607351124287, 0.023883111774921417, -0.0065996372140944, -0.03521609678864479, -0.017621571198105812, 0.01887928880751133, 0.031185990199446678, 0.021827487275004387, 0.01129241194576025, 0.005663110408931971, 0.01722938008606434, -0.00519653782248497, 0.01737814210355282, -0.03272770717740059, -0.0006571743870154023, 0.008357737213373184, -0.007208209950476885, -0.01858176477253437, -0.0425189733505249, 0.002968484302982688, -0.003763010259717703, -0.0075530679896473885, 0.016188044100999832, 0.02570883184671402, -0.035351336002349854, 0.041004300117492676, 0.005193157121539116, -0.0031983896624296904, -0.006592874880880117, 0.005798349156975746, 0.02269301377236843, 0.017472809180617332, 0.02125948667526245, -0.03857000917196274, -0.005673253443092108, 0.0021857917308807373, -0.011623745784163475, 0.00370891485363245, -0.011245078407227993, -0.01391603797674179, -0.014186514541506767, -0.030131129547953606, 0.0072149718180298805, -0.007336686830967665, 0.0064711603336036205, 0.016701949760317802, 0.009831836447119713, 0.0070459237322211266, -0.003925296477973461, -0.013679370284080505, -0.022152060642838478, 0.02641207166016102, -0.01866290718317032, -0.010812315158545971, -0.004993680398911238, 0.015214326791465282, 0.00982507411390543, -0.04403364285826683, 0.0045981076546013355, 0.00671120872721076, -0.014308229088783264, -0.01016993261873722, -0.01616099663078785, 0.0003716944484040141, -0.005027489736676216, -0.009135358035564423, 0.016350330784916878, -0.02937379479408264, -0.008499737828969955, 0.02750750258564949, 0.031240085139870644, -0.006055301986634731, 0.0044932980090379715, -0.01913624256849289])
print(simsearch_by_vector_with_score)

[(Document(metadata={'id': 8, 'summary': 'Nasty No flavor'}, page_content='The candy is just red , No flavor . Just  plan and chewy .  I would never buy them again'), 0.9648153551769503), (Document(metadata={'id': 4, 'summary': 'stale product'}, page_content='Arrived in 6 days and were so stale i could not eat any of the 6 bags!!'), 0.9655108580341948), (Document(metadata={'id': 3, 'summary': 'Taste is neutral, quantity is DECEITFUL!'}, page_content='The taste of these white cheddar flat breads is like a regular cracker - which is not bad, except that I bought them because I wanted a cheese taste.<br /><br />What was a HUGE disappointment? How misleading the packaging of the box is. The photo on the box (I bought these in store) makes it look like it is full of long flatbreads (expanding the length and width of the box). Wrong! The plastic tray that holds the crackers is about 2 smaller all around - leaving you with about 15 or so small flatbreads.<br /><br />What is also bad about thi

## Delete Row by ID

In [35]:
# delete row by id
vector_store.delete(["3", "7"])

True

## Drop Vector Store

In [ ]:
# drop vectorstore
vector_store.drop()

# Load a Document from Azure Blob Storage

Below is example of loading a file from Azure Blob Storage container into the SQL Vector store after splitting the document into chunks.
[Azure Blog Storage](https://learn.microsoft.com/en-us/azure/storage/blobs/storage-blobs-introduction) is Microsoft's object storage solution for the cloud. Blob Storage is optimized for storing massive amounts of unstructured data. 

In [ ]:
pip install azure-storage-blob

In [36]:
from langchain.document_loaders import AzureBlobStorageFileLoader
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define your connection string and blob details
conn_str = "DefaultEndpointsProtocol=https;AccountName=<YourBlobName>;AccountKey=<YourAccountKey>==;EndpointSuffix=core.windows.net"
container_name = "<YourContainerName"
blob_name = "01 Harry Potter and the Sorcerers Stone.txt"

# Create an instance of AzureBlobStorageFileLoader
loader = AzureBlobStorageFileLoader(conn_str=conn_str, container=container_name, blob_name=blob_name)

# Load the document from Azure Blob Storage
documents = loader.load()

# Split the document into smaller chunks if necessary
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)

# Print the number of split documents
print(f"Number of split documents: {len(split_documents)}")


Number of split documents: 528


API Reference:[AzureBlobStorageContainerLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.azure_blob_storage_container.AzureBlobStorageContainerLoader.html)

In [39]:
# # Initialize the vector store & insert the documents in AzureSQLDB with their embeddings
vector_store = SQLServer_VectorStore(
    connection_string=_CONNECTION_STRING,
    distance_strategy=DistanceStrategy.COSINE,
    embedding_function=embeddings,
    embedding_length=1536,
    table_name="harrypotter"
)  # Replace with your actual vector store initialization

# Add split documents to the vector store individually
for i, doc in enumerate(split_documents):
    vector_store.add_documents(documents=[doc], ids=[f"doc_{i}"])

print("Documents added to the vector store successfully!")

Documents added to the vector store successfully!


In [46]:
from typing import List, Tuple

# Perform similarity search
query = "Why did the Dursleys not want Harry in their house?"
docs_with_score: List[Tuple[Document, float]] = vector_store.similarity_search_with_score(query)

for doc, score in docs_with_score:
    print("-" * 60)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 60)

------------------------------------------------------------
Score:  0.3626232679001803
The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn’t think they could bear it if anyone found out about the Potters. Mrs. Potter was Mrs. Dursley’s sister, but they hadn’t met for several years; in fact, Mrs. Dursley pretended she didn’t have a sister, because her sister and her good-for-nothing husband were as unDursleyish as it was possible to be. The Dursleys shuddered to think what the neighbors would say if the Potters arrived in the street. The Dursleys knew that the Potters had a small son, too, but they had never even seen him. This boy was another good reason for keeping the Potters away; they didn’t want Dudley mixing with a child like that.
------------------------------------------------------------
------------------------------------------------------------
Score:  0.44752797298657554
The Dursleys’

## API reference 

For detailed documentation of all \_\_ModuleName\_\_VectorStore features and configurations head to the API reference: [https://python.langchain.com/api\_reference/sqlserver/index.html](https:\python.langchain.com\api_reference\sqlserver\index.html)

## Related
- Vector store [conceptual guide](https://python.langchain.com/docs/concepts/vectorstores/)
- Vector store [how-to guides](https://python.langchain.com/docs/how_to/#vector-stores)